In [1]:
from streaming.base.format.mds.encodings import Encoding, _encodings
from streaming import MDSWriter, LocalDataset
from tqdm import tqdm
from typing import List
import torch
import json

class ListOfDict(Encoding):
    def encode(self, obj: List[dict]) -> bytes:
        # Convert the list of dictionaries to a JSON-encoded string
        json_str = json.dumps(obj)
        return json_str.encode('utf-8')

    def decode(self, data: bytes) -> List[dict]:

        # Decode the JSON-encoded string back to a list of dictionaries
        json_str = data.decode('utf-8')
        return json.loads(json_str)

# Register the custom encoding for 'list_of_dict'
_encodings['list_of_dict'] = ListOfDict

In [5]:
# !wget https://huggingface.co/datasets/mesolitica/translated-LLaVA-Pretrain/resolve/main/blip_laion_cc_sbu_558k.translated.jsonl

In [9]:
import json
import os

In [4]:
roles = {
    'human': 'user',
    'gpt': 'assistant'
}

In [16]:
data_en, data_ms = [], []
with open('blip_laion_cc_sbu_558k.translated.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        f = os.path.join('/home/ubuntu/filtered-blip-images', l['image'])
        if not os.path.exists(f):
            continue
        en, ms = [], []
        for c in l['conversations']:
            en_ = c['value'].replace('<imej>','<image>').replace('<img>','<image>').replace('<gambar>','<image>')
            ms_ = c['value_ms'].replace('<imej>','<image>').replace('<img>','<image>').replace('<gambar>','<image>')
            en.append({
                'role': roles[c['from']],
                'content': en_
            })
            ms.append({
                'role': roles[c['from']],
                'content': ms_
            })
        
        data_en.append({'filename': [f], 'conversations': en})
        data_ms.append({'filename': [f], 'conversations': ms})

558128it [00:08, 69253.83it/s]


In [17]:
len(data_en), len(data_ms)

(238135, 238135)

In [19]:
columns = {
    'conversations': 'list_of_dict',
    'filename': 'list_of_dict'
}

hashes = 'sha1', 'xxh64'

In [20]:
!rm -rf mosaic-vision

In [21]:
data = data_en + data_ms

In [22]:
with MDSWriter(out='mosaic-vision', columns=columns, compression=None, hashes=hashes) as out:
    for l in tqdm(data):
        for i in range(len(l['conversations'])):
            l['conversations'][i]['content'] = l['conversations'][i]['content'].replace('\n<image>', ' <image>').replace('<image>\n', '<image>').replace('\n<audio>', ' <audio>').replace('<audio>\n', '<audio>').strip()
            l['conversations'][i]['content'] = l['conversations'][i]['content'].replace('<image>', '<image> </image>').replace('<audio>', '<audio> </audio>')
        out.write(l)

100%|██████████| 476270/476270 [00:05<00:00, 82383.25it/s]
